In [1]:
!ls /kaggle/input/processed-datasets/processed_dataset1/kaggle/working/data/processed/dataset1

test  train  train_aug	valid


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNBranch(nn.Module):
    """
    CNN branch to extract local features.
    Input: (batch, 3, 640, 640)
    Output: (batch, 128) after global average pooling.
    """
    def __init__(self):
        super(CNNBranch, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)  # (32, 320, 320)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)  # (64, 160, 160)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)  # (128, 80, 80)
        )
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        return x.view(x.size(0), -1)

# Quick test:
dummy_input = torch.randn(4, 3, 640, 640)
cnn_model = CNNBranch()
print("CNN Branch Output Shape:", cnn_model(dummy_input).shape)  # Expected: [4, 128]


CNN Branch Output Shape: torch.Size([4, 128])


In [2]:
import torch
import torch.nn as nn

class ViTBranch(nn.Module):
    """
    ViT branch to extract global features.
    Splits the image into patches, embeds them, adds positional encoding,
    and processes them with Transformer encoder layers.
    Input: (batch, 3, 640, 640)
    Output: (batch, 768)
    """
    def __init__(self, image_size=640, patch_size=16, in_channels=3, embed_dim=768, num_layers=7, num_heads=8):
        super(ViTBranch, self).__init__()
        self.patch_size = patch_size
        num_patches = (image_size // patch_size) ** 2  # 1600 patches for 640x640
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches, embed_dim))
        # Use batch_first=True to improve performance and debugging
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
    
    def forward(self, x):
        x = self.proj(x)  # (batch, embed_dim, 40, 40)
        x = x.flatten(2).transpose(1, 2)  # (batch, 1600, embed_dim)
        x = x + self.pos_embedding
        x = self.transformer(x)
        x = x.mean(dim=1)
        return x

# Quick test:
vit_model = ViTBranch()
print("ViT Branch Output Shape:", vit_model(dummy_input).shape)  # Expected: [4, 768]


ViT Branch Output Shape: torch.Size([4, 768])


In [3]:
import torch
import torch.nn as nn

class HybridImagingModel(nn.Module):
    """
    Hybrid Imaging Model combining the CNN and ViT branches.
    Uses simple concatenation of CNN (128-dim) and ViT (768-dim) outputs,
    then projects to a 512-dim feature vector.
    """
    def __init__(self):
        super(HybridImagingModel, self).__init__()
        self.cnn_branch = CNNBranch()  # Defined in Cell 1
        self.vit_branch = ViTBranch()    # Defined in Cell 2
        self.fc = nn.Linear(128 + 768, 512)
    
    def forward(self, x):
        cnn_features = self.cnn_branch(x)
        vit_features = self.vit_branch(x)
        fused_features = torch.cat((cnn_features, vit_features), dim=1)
        out = self.fc(fused_features)
        return out

# Quick test:
hybrid_model = HybridImagingModel()
print("Hybrid Model Output Shape:", hybrid_model(dummy_input).shape)  # Expected: [4, 512]


Hybrid Model Output Shape: torch.Size([4, 512])


In [4]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import cv2
from pathlib import Path

# --- Define Utility and Dataset classes (reuse from above) ---

def get_unique_labels(labels_dir):
    labels_dir = Path(labels_dir)
    unique_labels = set()
    for label_file in labels_dir.glob("*.txt"):
        with open(label_file, "r") as f:
            line = f.readline().strip()
            if line:
                try:
                    unique_labels.add(int(line.split()[0]))
                except Exception as e:
                    print(f"Error reading label from {label_file}: {e}")
    return unique_labels

class ProcessedImagingDataset(Dataset):
    """
    Custom dataset for preprocessed images and labels.
    Filters out images with missing or empty label files.
    """
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = Path(images_dir)
        self.labels_dir = Path(labels_dir)
        all_images = list(self.images_dir.glob("*.jpg"))
        # Filter images that have a corresponding non-empty label file
        self.image_files = []
        for img in all_images:
            label_file = self.labels_dir / (img.stem + ".txt")
            if label_file.exists() and label_file.stat().st_size > 0:
                self.image_files.append(img)
        self.transform = transform
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = cv2.imread(str(img_path))
        if img is None:
            raise RuntimeError(f"Unable to read image: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        
        label_file = self.labels_dir / (img_path.stem + ".txt")
        with open(label_file, "r") as f:
            line = f.readline().strip()
            if line:
                label = int(line.split()[0])
            else:
                raise ValueError(f"No label found in {label_file}")
        
        label = torch.tensor(label, dtype=torch.long)
        if label.item() < 0 or label.item() > 8:
            raise ValueError(f"Label {label.item()} from {label_file} is out of expected range [0, 8].")
        return img, label

# ImagingClassifier definition (using HybridImagingModel from Cell 3)
class ImagingClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ImagingClassifier, self).__init__()
        self.hybrid_model = HybridImagingModel()  # Defined in Cell 3
        self.classifier = nn.Linear(512, num_classes)
    
    def forward(self, x):
        features = self.hybrid_model(x)
        logits = self.classifier(features)
        return logits

def train_model(model, dataloader, criterion, optimizer, device, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)
            total += labels.size(0)
        
        epoch_loss = running_loss / total
        epoch_acc = correct.double() / total
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")
    return model

# --- Main Training Loop ---
if __name__ == "__main__":
    # Set paths for Dataset1 processed data.
    images_dir = "/kaggle/input/processed-datasets/processed_dataset1/kaggle/working/data/processed/dataset1/train/images"
    labels_dir = "/kaggle/input/processed-datasets/processed_dataset1/kaggle/working/data/processed/dataset1/train/labels"
    
    unique_labels = get_unique_labels(labels_dir)
    print("Unique labels in training set:", unique_labels)
    num_classes = len(unique_labels)
    
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize((640, 640)),
        T.ToTensor(),
    ])
    
    train_dataset = ProcessedImagingDataset(images_dir, labels_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    
    # Test forward pass on CPU.
    dummy_img, dummy_label = train_dataset[0]
    dummy_img = dummy_img.unsqueeze(0)
    model_cpu = ImagingClassifier(num_classes=num_classes)
    try:
        test_out = model_cpu(dummy_img)
        print("Forward pass on CPU successful, output shape:", test_out.shape)
    except Exception as e:
        print("Error during forward pass on CPU:", e)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ImagingClassifier(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    
    print("Starting training of the Imaging Module on Dataset1...")
    trained_model = train_model(model, train_loader, criterion, optimizer, device, num_epochs=20)
    
    torch.save(trained_model.state_dict(), "trained_imaging_model.pth")
    print("Training completed and model saved as trained_imaging_model.pth")


In [5]:
# Cell: Training Script for Imaging Module on Dataset2

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import cv2
from pathlib import Path

# --- Utility: Get Unique Labels from the Training Set ---
def get_unique_labels(labels_dir):
    labels_dir = Path(labels_dir)
    unique_labels = set()
    for label_file in labels_dir.glob("*.txt"):
        with open(label_file, "r") as f:
            line = f.readline().strip()
            if line:
                try:
                    unique_labels.add(int(line.split()[0]))
                except Exception as e:
                    print(f"Error reading label from {label_file}: {e}")
    return unique_labels

# --- Custom Dataset for Processed Imaging Data ---
class ProcessedImagingDataset(Dataset):
    """
    Custom dataset for loading preprocessed images and labels.
    Filters out images with missing or empty label files.
    """
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = Path(images_dir)
        self.labels_dir = Path(labels_dir)
        all_images = list(self.images_dir.glob("*.jpg"))
        # Filter images that have a corresponding non-empty label file
        self.image_files = [img for img in all_images 
                            if (self.labels_dir / (img.stem + ".txt")).exists() 
                            and (self.labels_dir / (img.stem + ".txt")).stat().st_size > 0]
        self.transform = transform
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = cv2.imread(str(img_path))
        if img is None:
            raise RuntimeError(f"Unable to read image: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        
        label_file = self.labels_dir / (img_path.stem + ".txt")
        with open(label_file, "r") as f:
            line = f.readline().strip()
            if line:
                label = int(line.split()[0])
            else:
                raise ValueError(f"No label found in {label_file}")
        label = torch.tensor(label, dtype=torch.long)
        if label.item() < 0:
            raise ValueError(f"Label {label.item()} from {label_file} is negative.")
        return img, label

# --- ImagingClassifier Definition ---
# Assumes HybridImagingModel is already defined (from Cell 3)
class ImagingClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ImagingClassifier, self).__init__()
        self.hybrid_model = HybridImagingModel()  # Defined in earlier cell
        self.classifier = nn.Linear(512, num_classes)
    
    def forward(self, x):
        features = self.hybrid_model(x)
        logits = self.classifier(features)
        return logits

# --- Training Function ---
def train_model(model, dataloader, criterion, optimizer, device, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)
            total += labels.size(0)
        
        epoch_loss = running_loss / total
        epoch_acc = correct.double() / total
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")
    return model

# --- Main Training Loop for Dataset2 ---
if __name__ == "__main__":
    # Set paths for processed Dataset2 (update if needed)
    images_dir = "/kaggle/input/processed-datasets/processed_dataset2/kaggle/working/data/processed/dataset2/train/images"
    labels_dir = "/kaggle/input/processed-datasets/processed_dataset2/kaggle/working/data/processed/dataset2/train/labels"
    
    unique_labels = get_unique_labels(labels_dir)
    print("Unique labels in training set (Dataset2):", unique_labels)
    num_classes = len(unique_labels)
    
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize((640, 640)),
        T.ToTensor(),
    ])
    
    train_dataset = ProcessedImagingDataset(images_dir, labels_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    
    # Test a forward pass on CPU
    dummy_img, dummy_label = train_dataset[0]
    dummy_img = dummy_img.unsqueeze(0)
    model_cpu = ImagingClassifier(num_classes=num_classes)
    try:
        test_out = model_cpu(dummy_img)
        print("Forward pass on CPU successful, output shape:", test_out.shape)
    except Exception as e:
        print("Error during forward pass on CPU:", e)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ImagingClassifier(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    
    print("Starting training of the Imaging Module on Dataset2...")
    trained_model = train_model(model, train_loader, criterion, optimizer, device, num_epochs=20)
    
    torch.save(trained_model.state_dict(), "trained_imaging_model_dataset2.pth")
    print("Training completed and model saved as trained_imaging_model_dataset2.pth")


Unique labels in training set (Dataset2): {0, 1, 2, 3, 4}
Forward pass on CPU successful, output shape: torch.Size([1, 5])
Starting training of the Imaging Module on Dataset2...
Epoch 1/20 - Loss: 3.0570 - Accuracy: 0.2474
Epoch 2/20 - Loss: 1.5775 - Accuracy: 0.3299
Epoch 3/20 - Loss: 1.4627 - Accuracy: 0.3711
Epoch 4/20 - Loss: 1.5724 - Accuracy: 0.4021
Epoch 5/20 - Loss: 1.7034 - Accuracy: 0.3093
Epoch 6/20 - Loss: 1.2537 - Accuracy: 0.4845
Epoch 7/20 - Loss: 1.3138 - Accuracy: 0.4124
Epoch 8/20 - Loss: 1.5740 - Accuracy: 0.4433
Epoch 9/20 - Loss: 1.6407 - Accuracy: 0.3918
Epoch 10/20 - Loss: 1.3514 - Accuracy: 0.4639
Epoch 11/20 - Loss: 1.4251 - Accuracy: 0.4536
Epoch 12/20 - Loss: 1.2154 - Accuracy: 0.5052
Epoch 13/20 - Loss: 1.1211 - Accuracy: 0.6186
Epoch 14/20 - Loss: 1.1916 - Accuracy: 0.4948
Epoch 15/20 - Loss: 1.1331 - Accuracy: 0.5258
Epoch 16/20 - Loss: 1.2061 - Accuracy: 0.4845
Epoch 17/20 - Loss: 1.1286 - Accuracy: 0.5567
Epoch 18/20 - Loss: 1.2185 - Accuracy: 0.4845
Epo

In [ ]:
!ls 